# QC: Results match Capacity Assumptions from TWG meeting and Board

Board of Directors Meeting MAY 25, 2018

### 509,000 capacity

https://www.sandag.org/uploads/meetingid/meetingid_4785_23865.pdf (page 852)

In [ ]:
import os
import sys

In [ ]:
# append path to find utils module in urbansim
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
parentdir2 =  os.path.abspath(os.path.join(parentdir, os.pardir))
sys.path.append(parentdir2) # to get path to utils module

In [ ]:
import pandas as pd
import numpy as np
import utils

In [ ]:
from sqlalchemy import create_engine
from database import get_connection_string

In [ ]:
%matplotlib inline

In [ ]:
# connect to database
db_connection_string = get_connection_string('..\..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
# get versions
versions = utils.yaml_to_dict('../../data/scenario_config.yaml', 'scenario')

### Get board report capacity assumptions

In [ ]:
d = {'jur_id': range(1,20),\
    'twg2016to2035': [6600, 27700,200,100,3200,2500,11800,3600,9300,1100,5800,6900,1500,202600,11500,3300,400,7100,67800],\
     'twg2036to2050': [2100, 14200,200,200,7400,1100,8800,300,1900,1500,6400,5200,2000,65600,4300,3400,200,3400,7800]}
dfboard = pd.DataFrame(data=d)
#dfboard['jur_id'] = range(1,20)
# dfboard

In [ ]:
dfboard['twgTotal'] = dfboard.iloc[:, 1:3].sum(1)

In [ ]:
# dfboard

# Get geographies

In [ ]:
lookup_sql = '''
SELECT parcel_id,jur_id
FROM [isam].[xpef04].[parcel2015_mgra_jur_cpa] 
WHERE  i=1'''
lookup_df = pd.read_sql(lookup_sql, mssql_engine)
jur_name_sql = '''SELECT [jurisdiction_id] as jur_id,[name] as jur_name FROM [urbansim].[ref].[jurisdiction]'''
jur_name = pd.read_sql(jur_name_sql,mssql_engine)
lookup_df = pd.merge(lookup_df,jur_name,on='jur_id',how='left')

# Get units added

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

In [ ]:
# run_id = input()

In [ ]:
output_sql = '''
    SELECT parcel_id,year_simulation,unit_change
    FROM urbansim.urbansim.urbansim_lite_output urbansim_lite_output 
    WHERE  run_id =  %s'''
output_sql = output_sql % run_id
output = pd.read_sql(output_sql,mssql_engine)

In [ ]:
print("units added sum: {:,}".format(output.unit_change.sum()))

## Classify units into two categories based on year added: "2016 to 2035" & "2036 to 2050"

In [ ]:
output['yr_category'] = 'urb2016to2035'

In [ ]:
output.loc[output['year_simulation']>2035,'yr_category'] = 'urb2036to2050' 

##  Add city to output dataframe

In [ ]:
output = pd.merge(output,lookup_df,on='parcel_id',how='left')

In [ ]:
# output.head()

## Group by jurisdiction and year category and sum unit change

In [ ]:
df = pd.DataFrame({'unit_change': output.groupby(["jur_name","jur_id","yr_category"])
                                          .unit_change.sum()}).reset_index()

In [ ]:
df.unit_change.sum()
#419,558

In [ ]:
# df.head()

## Pivot data so year category is a column and jurisdiction is a row

In [ ]:
table = pd.pivot_table(df, values='unit_change', index=[ 'jur_id','jur_name'],columns=['yr_category'])

In [ ]:
table.reset_index(inplace=True)

In [ ]:
table.columns.name = None

In [ ]:
# table

In [ ]:
table['urbTotal'] = table.iloc[:, 2:4].sum(1)

In [ ]:
table.urbTotal.sum()
#419558

In [ ]:
table_w_boardreport = pd.merge(dfboard,table,on='jur_id',how='left')

In [ ]:
table_w_boardreport.loc['Total']= table_w_boardreport.sum()

In [ ]:
table_w_boardreport.loc[table_w_boardreport.jur_id==190,'jur_name'] = 'Region'

In [ ]:
table_w_boardreport['jur_id'] = table_w_boardreport['jur_id'].astype('int')

In [ ]:
table_w_boardreport.loc[table_w_boardreport.jur_id==190,'jur_id'] = 'Total'

In [ ]:
table_w_boardreport.set_index(['jur_id'],inplace=True)

In [ ]:
# table_w_boardreport

In [ ]:
t2016to2035 = table_w_boardreport[['jur_name','twg2016to2035','urb2016to2035']].copy()

In [ ]:
t2016to2035['remaining2016to2035'] = t2016to2035['twg2016to2035'] - t2016to2035['urb2016to2035']

In [ ]:
# t2016to2035

In [ ]:
num_remaining2016to2035 = len(t2016to2035.loc[t2016to2035.remaining2016to2035<0])

In [ ]:
t2036to2050 = table_w_boardreport[['jur_name','twg2036to2050','urb2036to2050']].copy()

In [ ]:
t2036to2050['remaining2036to2050'] = t2036to2050['twg2036to2050'] - t2036to2050['urb2036to2050']

In [ ]:
t2016to2050 = table_w_boardreport[['jur_name','twg2016to2035','urb2016to2035','twg2036to2050','urb2036to2050']].copy()
t2016to2050['remaining2016to2035'] = t2016to2050['twg2016to2035'] - t2016to2050['urb2016to2035']
t2016to2050['remaining2036to2050'] = t2016to2050['twg2036to2050'] - t2016to2050['urb2036to2050']

In [ ]:
# t2036to2050

In [ ]:
num_remaining2036to2050 = len(t2036to2050.loc[t2036to2050.remaining2036to2050<0])

In [ ]:
tTotal = table_w_boardreport[['jur_name','twgTotal','urbTotal']].copy()

In [ ]:
tTotal['remaining'] = tTotal['twgTotal'] - tTotal['urbTotal']

In [ ]:
# tTotal

In [ ]:
num_remaining = len(tTotal.loc[tTotal.remaining<0])

In [ ]:
print('From Board Report with 509,000 capacity')
table_w_boardreport[['jur_name','twg2016to2035','twg2036to2050','twgTotal','urb2016to2035','urb2036to2050','urbTotal']]

#### Note: over the entire forecast period all cities are below the capacity assumptions in the board report, except Unincorporated

#### Remaining capacity in the first increment (2016 to 2035) is used in the second increment (2036 to 2050)

#### Thus the total capacity used in the later period (2036 to 2050) is greater than the board report

In [ ]:
if (num_remaining + num_remaining2036to2050 + num_remaining2016to2035) == 0:
    print("for run id : {:,}\n".format(run_id))
    print('\n\nQC Pass: Urbansim units matches board report capacity assumptions\n\n')
else:
    print("\nfor run id : {:,}\n".format(run_id))
    print('\n\nQC Fail: Urbansim units are not less than the board report capacity assumptions\n\n')
    if num_remaining2016to2035 > 0:
        print('\n\nFailed Cities 2016 to 2035:\n\n')
        print(t2016to2035.loc[t2016to2035.remaining2016to2035<0])
    if num_remaining2036to2050 > 0:
        print('\n\nFailed Cities 2036 to 2050:\n\n')
        print(t2016to2050.loc[t2016to2050.remaining2036to2050<0])
    if num_remaining > 0:
        print('\n\n\n\nFailed Cities Total over entire forecast: 2016 to 2050:\n\n')
        print(tTotal.loc[tTotal.remaining<0])
    print('\n\n\nQC Fail\n\n')